# 装饰器 Decorator

In [362]:
def func():
    print("Hello,world!")
    return 
    
func()

Hello,world!


In [363]:
import time 

def time_counter():
    start=time.time()
    print("Hello,world!")
    end=time.time()
    return end-start



In [364]:
time_counter()

Hello,world!


0.0

In [365]:
def time_counter2(fn):
    start=time.time()
    fn()
    end=time.time()
    return end-start


In [366]:
time_counter2(func)

Hello,world!


0.0

In [367]:
import functools
from functools import update_wrapper

# 改进一下
def time_counter3(fn):
    @functools.wraps(func)
    def wrapper(*args,**kwargs):
        start=time.time()
        result=fn(*args,**kwargs)
        end=time.time()
        print(f"{fn.__name__} execution time: {end-start} s")
        return result
    
    update_wrapper(wrapper,fn)
    return wrapper

In [368]:
func=time_counter3(func)
func()

Hello,world!
func execution time: 0.0 s


In [369]:
# 如果直接输出的话就是新函数的信息
time_counter3(func)

<function __main__.func()>

In [370]:
# 直接装饰
@time_counter3
def func_1():
    print("成都永远的红")
    
func_1()

成都永远的红
func_1 execution time: 0.0011301040649414062 s


## 2 利用装饰器为函数加入日志功能

In [371]:
from datetime import datetime,timezone

def logger(fn):
    def inner(*args,**kwargs):
        called_at=datetime.now()
        to_execute=fn(*args,**kwargs)
        print(f"{fn.__name__} executed. Logged at {called_at}")
        return to_execute # 一定要返回函数的执行结果，否则导致装饰后的函数出错
    
    return inner

In [372]:
@logger
def func_2():
    print("不惧暴雨和狂风")

func_2()

不惧暴雨和狂风
func_2 executed. Logged at 2024-12-12 20:56:16.145028


## 3


In [373]:
@logger
@time_counter3
def foo(x):
    sum_=0
    for i in range(x):
        sum_+=i**2
    return sum_

a=foo(2) #左闭右开区间，所以只有1

foo execution time: 0.0 s
foo executed. Logged at 2024-12-12 20:56:16.189597


In [374]:
a

1

In [375]:
foo(1000)

foo execution time: 0.0 s
foo executed. Logged at 2024-12-12 20:56:16.272730


332833500

## 4 带参数的装饰器

- 带参数的装饰器，其实是在装饰器函数的外面又包裹了一个函数，使得该函数接收参数，返回的是装饰器的函数

In [376]:
def logging(flag):
    def decorator(fn):
        def inner(num1,num2):
            if flag=="+":
                print("正在加")
            elif flag=="-":
                print("正在减")
            result=fn(num1,num2)
            print(f"result:{result}")
            return result
        
        return inner
    
    return decorator


In [377]:
@logging("+")
def add(a,b):
    return a+b

@logging("-")
def sub(a,b):
    return a-b

add(1,3)
sub(1,3)

正在加
result:4
正在减
result:-2


-2

## 5 多个装饰器

In [378]:
def make_bold(fn):
    def wrapped():
        return "<b>"+fn()+"</b>"
    
    return wrapped

def make_italic(fn):
    def wrapped():
        return "<i>"+fn()+"</i>"
    
    return wrapped

@make_bold
@make_italic #先调用，在里面一层
def func_3():
    return "热血流在心中"
    
func_3()
        

'<b><i>热血流在心中</i></b>'

## 6 多个带参数函数装饰器

In [379]:
def make_html_tag(tag,*args,**kwargs):
    def real_decorator(fn):
        css_class="class='{0}'".format(kwargs["css_class"] if "css_class" in kwargs else "")
        
        def wrapped(*args,**kwargs):
            return "<"+tag+css_class+">"+fn(*args,**kwargs)+"</"+tag+">"
        
        return wrapped
    
    return real_decorator

@make_html_tag(tag="b",css_class="bold_css")
@make_html_tag(tag="i",css_class="bold_i")
def func_4():
    return "坚持最初的梦"

func_4()

"<bclass='bold_css'><iclass='bold_i'>坚持最初的梦</i></b>"

## 7 类装饰器

In [380]:
class Decorator:
    def __init__(self,fn):
        print("inside Decorator.__init__()")
        self.fn=fn
        
    def __call__(self,*args,**kwargs):
        self.fn(*args,**kwargs)
        print("inside Decorator.__call__()")
        
@Decorator
def func_5():
    print("南看台的战士永不怕失败")

inside Decorator.__init__()


In [381]:
func_5()

南看台的战士永不怕失败
inside Decorator.__call__()


## 8 多个带参数类装饰器

In [382]:
class MakeHtmlTag:
    def __init__(self,tag,css_class="",*args,**kwargs):
        self._tag=tag
        self._css_class=" class='{0}'".format(css_class) if css_class!="" else ""
    
    def __call__(self,fn,*args,**kwargs):
        def wrapped(*args,**kwargs):
            return "<"+self._tag+self._css_class+">"+fn(*args,**kwargs)+"</"+self._tag+">"
        
        return wrapped
    

In [383]:
@MakeHtmlTag(tag="b",css_class="bold_css")
@MakeHtmlTag(tag="i",css_class="bold_i")
def func_6(name):
    return "Hello,{}!".format(name)

func_6('Ganchao')

"<b class='bold_css'><i class='bold_i'>Hello,Ganchao!</i></b>"

## 9 装饰类

In [384]:
def refac_str(cls):
    def __str__(self):
        return str(self.__dict__)
    
    cls.__str__ = __str__
    return cls

In [385]:
@refac_str
class MyClass:
    def __init__(self,x,y):
        self.x=x
        self.y=y
        

In [386]:
print(MyClass(39,11))

{'x': 39, 'y': 11}


## 10 装饰器的副作用

In [387]:
@time_counter3
def func_7():
    print("高举拳头创造我们的时代")
    
func_7()

高举拳头创造我们的时代
func_7 execution time: 0.0 s


In [388]:
func_7.__name__

'func_7'

- 解决方法：引入装饰器的内部函数，@wraps(func)

## 11 装饰器的应用案例

In [389]:
def fib(n):
    print(f"calculating term -{n}")
    if n<3:
        return 1
    else:
        return fib(n-1)+fib(n-2)

In [390]:
print(f"6th:{fib(6)}")

calculating term -6
calculating term -5
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
calculating term -3
calculating term -2
calculating term -1
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
6th:8


In [391]:
def memorizer(fn):
    cache={}
    
    def inner(n):
        if n not in cache:
            cache[n]=fn(n)
        
        return cache[n]
    
    return inner

In [392]:
@memorizer
def fib_1(n):
    print(f"calculating term -{n}")
    if n<3:
        return 1
    else:
        return fib(n-1)+fib(n-2)

In [393]:
print(f"8th:{fib_1(8)}")

calculating term -8
calculating term -7
calculating term -6
calculating term -5
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
calculating term -3
calculating term -2
calculating term -1
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
calculating term -5
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
calculating term -3
calculating term -2
calculating term -1
calculating term -6
calculating term -5
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
calculating term -3
calculating term -2
calculating term -1
calculating term -4
calculating term -3
calculating term -2
calculating term -1
calculating term -2
8th:21
